In [1]:
import pandas as pd
import numpy as np 
import re
from datetime import datetime
from stop_words import get_stop_words
from uk_stemmer import UkStemmer
import tokenize_uk

import matplotlib.pyplot as plt 
from nltk import stem
import pymorphy2
import razdel
import pickle
import six
import tomotopy as tp

In [2]:
import pyLDAvis

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/Users/oksana/Dev/TextClassification/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
news_filepath = '../data/april.csv'
news = pd.read_csv(news_filepath, index_col=[0])
# news = news.set_index('hash', drop=False)
news.columns

Index(['title', 'text', 'subtitle', 'link', 'domain', 'datetime', 'views',
       'created_at', 'category', 'language', 'id.1', 'domain_alias',
       'mycategory', 'mentions', 'hash', 'sentiment', 'sent_list', 'ment_tone',
       'topic'],
      dtype='object')

In [5]:
news['all_text'] = news.title.str.cat(news.text, sep='\n', na_rep = '')
news['all_text'] = news.all_text.str.strip()

In [5]:
news['datetime'] = pd.to_datetime(news.datetime, utc=True).dt.tz_convert('Europe/Kiev')

In [145]:
news.topic.update(part.topic)

In [2]:
# after topcs update save news to file (all columns but all_text)
news[['title', 'text', 'subtitle', 'link', 'domain', 'datetime', 'views',
       'created_at', 'category', 'language', 'id.1', 'domain_alias',
       'mycategory', 'mentions', 'hash', 'sentiment', 'sent_list', 'ment_tone',
       'topic']].to_csv('../data/april.csv')

In [37]:
news.topic.value_counts()

інциденти_решта                                                               17164
політика_решта                                                                13362
ковід_решта                                                                   12270
економіка_решта                                                                9617
міжн_пол_решта                                                                 8045
Стягнення Росією військ до кордону і реакція міжнародної спільноти             2649
Курс валют                                                                     1376
Довибори на Прикарпатті (87 округ)                                              870
Засідання РНБО і санкції, зокрема щодо контрабандистів                          727
Дипломатичний конфлікт між Чехією та Росією                                     707
Зв'язок між вакцинами (зокрема AstraZeneca та Johnson&Johnson) і тробмозом      705
Співпраця України з НАТО, заяви щодо потенційного членства в альянсі        

In [11]:
news['topic'] = None

In [6]:
news.groupby(['topic']).size()

topic
бюджет                            427
вагони/уз/інфрастуктура           237
вакцина/ліки                     1941
вибори                           4049
газ/тарифи/опалювальний сезон     825
дослідження/здоро'я               464
дтп                              2663
економіка_решта                  6557
енергетика                        573
ковід в україні                  3638
ковід політика                   1128
ковід у світі                    2539
ковід_решта                      4776
крим/росія/міжн_пол               523
ксу                              2923
курс валют                       1304
мвф/нбу                           450
медична система                  2227
міжнародна політика               614
опитування                        274
пенсії/зарплати/соцвиплати        638
пожежі                            706
політика_решта                   9072
порушення карантину                93
порушення на виборах              715
правопорушення                   3072
призна

In [39]:
news.mycategory.value_counts()

Політика               19763
Інциденти              18957
Коронавірус            15138
Закордонна політика    14403
Економіка              14081
Шоу-бізнес             13213
Світ                   13173
Суспільство            10189
Спорт                  10159
Війна/Донбас            8988
Життя/lifestyle         8803
Технології              5968
Карантин                4894
Погода                  2410
Інше                     500
Name: mycategory, dtype: int64

In [19]:
stop_words_ukr = get_stop_words('uk')
stop_words_ru = get_stop_words('ru')

additional_sw_ukr_path = '../dicts/more_stop_words_ukr.txt'
additional_sw_ru_path = '../dicts/more_stop_words_ru.txt'

with open(additional_sw_ukr_path) as f:
    more_stopwords_ukr = [w.strip() for w in f.readlines()]
    
with open(additional_sw_ru_path) as f:
    more_stopwords_ru = [w.strip() for w in f.readlines()]
    
stop_words_ukr += more_stopwords_ukr
stop_words_ru += more_stopwords_ru

stemmer_ru = stem.snowball.SnowballStemmer("russian") 
stemmer_ukr = UkStemmer()

stemmed_stopwords_ukr = list(set([stemmer_ukr.stem_word(word) for word in stop_words_ukr]))
stemmed_stopwords_ru = list(set([stemmer_ru.stem(word) for word in stop_words_ru]))

morph_ru = pymorphy2.MorphAnalyzer(lang='ru')
morph_ukr = pymorphy2.MorphAnalyzer(lang='uk')

In [32]:
category = 'Політика'
lang = 'uk'

In [33]:
ACCENT = six.unichr(769)
pat = re.compile(r'[а-яА-ЯіїІЇґҐєЄa-zA-Z\'`]{2,}')
WORD_TOKENIZATION_RULES = re.compile(r"""
[\w""" + ACCENT + """]+://(?:[a-zA-Z]|[0-9]|[$-_@.&+])+
|[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+
|[0-9]+-[а-яА-ЯіїІЇґҐєЄ'’`""" + ACCENT + """]+
|[+-]?[0-9](?:[0-9,.-]*[0-9])?
|[\w""" + ACCENT + """](?:[\w'’`-""" + ACCENT + """]?[\w""" + ACCENT + """]+)*
|[\w""" + ACCENT + """].(?:\[\w""" + ACCENT + """].)+[\w""" + ACCENT + """]?
|["#$%&*+,/:;<=>@^`~…\\(\\)⟨⟩{}\[\|\]‒–—―«»“”‘’'№]
|[.!?]+
|-+
""", re.X | re.U)

In [264]:
part = news[(news.language==lang)&(news.mycategory==category)].copy()
part.shape

(11918, 20)

In [34]:
part = news[(news.language==lang)&(news.mycategory==category)&(news.topic.str.contains('_решта'))].copy()
part.shape

(9456, 20)

In [119]:
# corpus_lemm = tp.utils.Corpus.load(corpus_lemm_path)
# corpus_stem = tp.utils.Corpus.load(corpus_stem_path)

### Українська

In [185]:
%%time
corpus_stem = tp.utils.Corpus(
    tokenizer = tp.utils.SimpleTokenizer(stemmer=stemmer_ukr.stem_word, pattern=WORD_TOKENIZATION_RULES), 
    stopwords = lambda x: x in stemmed_stopwords_ukr or not pat.match(x) 
)

corpus_stem.process(part.all_text)
# save preprocessed corpus for reuse
# corpus_stem.save(corpus_stem_path)

CPU times: user 6.25 s, sys: 62.4 ms, total: 6.31 s
Wall time: 6.65 s


727

In [90]:
%%time

lemm_exceptions = {
    'київ': 'київ',
    'києві': 'київ',
    'газу': 'газ',
    'газі': 'газ',
    'гроші': 'гроші',
    'народу': 'народ',
    'народом': 'народ',
    'народові': 'народ',
    'народі': 'народ',
    'рада': 'рада',
    'ради': 'рада',
    'раді': 'рада',
    'раду': 'рада',
    'банку': 'банк',
    'банки': 'банк',
    'набу': 'набу',
    'робота': 'робота',
    'роботи': 'робота',
    'енергетика': 'енергетика',
    'енергетики': 'енергетика',
    'рік': 'рік',
    'львова': 'львів'
}

def lemmatizer(word):
    if word in lemm_exceptions:
        return lemm_exceptions[word]
    return morph_ukr.parse(word)[0].normal_form


corpus_lemm = tp.utils.Corpus(
    tokenizer = tp.utils.SimpleTokenizer(stemmer=lemmatizer, pattern=WORD_TOKENIZATION_RULES), 
    stopwords = lambda x: x in stop_words_ukr or not pat.match(x) 
)

corpus_lemm.process(part.all_text)
# save preprocessed corpus for reuse
# corpus_lemm.save(corpus_lemm_path)

CPU times: user 4min, sys: 1.73 s, total: 4min 2s
Wall time: 4min 34s


3977

### Російська

In [23]:
%%time
# stemmed corpus
def tokenizer_ru(text, user_data):
    return [stemmer_ru.stem(_.text) for _ in razdel.tokenize(text)]

corpus_stem = tp.utils.Corpus(
    tokenizer = tokenizer_ru,
    stopwords = lambda x: x in stemmed_stopwords_ru or not pat.match(x) 
)

corpus_stem.process(part.all_text)
# save preprocessed corpus for reuse
# corpus_stem.save(corpus_stem_path)

CPU times: user 54.9 s, sys: 323 ms, total: 55.2 s
Wall time: 56.9 s


2474

In [86]:
%%time
# lemmatized corpus
def tokenizer_ru(text, user_data):
    return [morph_ru.parse(_.text)[0].normal_form for _ in razdel.tokenize(text)]

corpus_lemm = tp.utils.Corpus(
    tokenizer = tokenizer_ru,
    stopwords = lambda x: x in stop_words_ru or not pat.match(x) 
)


corpus_lemm.process(part.all_text)
corpus_lemm.save(corpus_lemm_path)

CPU times: user 1min 35s, sys: 362 ms, total: 1min 36s
Wall time: 1min 37s


### Тренування моделі

In [186]:
%%time
mdl = tp.LDAModel(min_df=3, rm_top=5, k=40, corpus=corpus_stem)
mdl.train(20)

print('Num docs:{}, Num Vocabs:{}, Total Words:{}'.format(
    len(mdl.docs), len(mdl.used_vocabs), mdl.num_words
))
print('Removed Top words: ', *mdl.removed_top_words)
 
# Let's train the model
for i in range(0, 1200, 20):
    print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))
    mdl.train(20)
print('Iteration: {:04}, LL per word: {:.4}'.format(1200, mdl.ll_per_word))

mdl.summary()

topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq

Num docs:727, Num Vocabs:4447, Total Words:116521
Removed Top words:  україн
Iteration: 0000, LL per word: -7.159
Iteration: 0020, LL per word: -7.073
Iteration: 0040, LL per word: -7.044
Iteration: 0060, LL per word: -7.023
Iteration: 0080, LL per word: -7.016
Iteration: 0100, LL per word: -7.016
Iteration: 0120, LL per word: -7.008
Iteration: 0140, LL per word: -7.008
Iteration: 0160, LL per word: -7.008
Iteration: 0180, LL per word: -7.011
Iteration: 0200, LL per word: -7.009
Iteration: 0220, LL per word: -7.01
Iteration: 0240, LL per word: -7.006
Iteration: 0260, LL per word: -7.014
Iteration: 0280, LL per word: -7.012
Iteration: 0300, LL per word: -7.015
Iteration: 0320, LL per word: -7.016
Iteration: 0340, LL per word: -7.013
Iteration: 0360, LL per word: -7.011
Iteration: 0380, LL per word: -7.015
Iteration: 0400, LL per word: -7.022
Iteration: 0420, LL per word: -7.019
Iteration: 0440, LL per word: -7.016
Iteration: 0460, LL per word: -7.018
Iteration: 0480, LL per word: -7.02


In [187]:
mdl.perplexity

1108.5252915574515

In [432]:
# mdl.save('lda_models/covid_uk_lemm.bin')

In [190]:
n_top_words = 30
top_words_dict = {}
for i, topic_dist in enumerate(topic_term_dists):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    top_words_dict[i]=' '.join(topic_words)
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: рнбо засіданн президент санкці рад зеленськ безпек оборон колишн національн питанн віктор володимир даніл рішенн міністр держав медведчук офіс янукович донбас крим секретар санкц пятниц російськ дружин микол українськ нацбезпек
Topic 1: он что это гражданств контрабанд из александр таможн контрабандист ег которы санкц украин альперин от списк сбу под год список но такж служб данил суд человек украинск таможенн об ест
Topic 2: снбо санкц украин заседан президент зеленск что решен апрел безопасност совет ввел оборон национальн лиц такж владимир вопрос контрабандист этом введен государств компан от об виктор из которы экс было
Topic 3: контрабанд санкці рнбо контрабандист зеленськ президент громадянств українськ осіб володимир олександр рад рішенн засіданн санкц альперін митниц митн сбу безпек оборон заяв топ десятк національн служб джерел списк зазнач назв
Topic 4: тов санкці указ ді санкц рішенн компан рнбо національн держав заход іноземн застосуванн осіб зеленськ володимир обм

In [189]:
# assign topics to news
topics = np.argmax(doc_topic_dists, axis=1)
part['topic_40_stem'] = topics
part['topic_40_stem'].value_counts()

3    272
0    169
2    168
1     68
4     50
Name: topic_40_stem, dtype: int64

In [ ]:
# save some news to file to scan each topic
res = []
column_name = 'topic_40_stem'
for topic in part[column_name].value_counts().index:
    res.append(part[part[column_name]==topic][[column_name, 'title', 'all_text']].sample(8))
res = pd.concat(res)
res.to_csv(f'topic_samples/{category}_{lang}_{column_name}.csv', index=False)

In [ ]:
# print 20 titles for selected topic
for t in part[part.topic_40_stem==1].title.sample(20):
    print(t)

#### Політика

In [139]:
# Політика - 40 тем
# uk
# 23 - скупчення російських військ біля кордону/можливість вторгнення/реакція Заходу
# 8 - вибори на Прикарпатті
# 24 - Путін/політика, що стосується Росії 
# 34 - засідання і санкції введені РНБО
# 26 - кадрові перестановки в Нафтогазі і міністерстві енергетики
# 9 - санкції проти каналів медведчука (їх критика)
# 17 - новини про декларації чиновників
# 15 - ліквідація ОАСК і інші судові розгляди
# 37 - зв'язки з молдовою і викрадення чауса
# 2 - вечірка тищенка в карантин
pol_topic_dict = {
#                     25: "КСУ та його відсторонений голова Тупицький",
#                     1:'Довибори на Прикарпатті (87 округ)', 
                    0:'Стягнення Росією військ до кордону',
                    1:'Стягнення Росією військ до кордону',
#                     4: "Співпраця України з НАТО, заяви щодо потенційного членства в альянсі",
                    2: "Співпраця України з НАТО, заяви щодо потенційного членства в альянсі"
#                     3: "Робота РНБО, засідання та санкції",
#                     24: "Зміна керівництва Нафтогазу й міністерства енергетики",
#                     13: "Огляд декларацій чиновників",
#                     35: "Вечірка нардепа Тищенка під час локдауну"
#                     7: "ОАСК та законопроект про його ліквідацію",
#                     9: "Результати соцопитувань",
#                     7: "Дисертація Іллі Киви"
#                     20: "Нардеп Шевченко та його візит у Білорусь"
#                     20: "Віктор Медведчук та санкції проти нього",
#                     21: "Тема Криму та кримських татар",
#                     1: "Події в Молодві і викрадення екссудді Чауса"
#                     7: "Санкції в ПАРЄ проти депутата Гончаренка"
                 }

part['topic'] = part.topic_40_stem.apply(lambda x: pol_topic_dict.get(x, 'політика_решта'))

#### Економіка

In [242]:
# 19 - курс валют
# 1 - пенсії на соцвиплати?
# 10 - ціна на газ
# 29 - карантинні виплати фопам
# 4 - імпорт електроенергії з росії
# 21 - екологічна модернізація екноміки?
# 24 - зелена енергетика?
# 37 - співпраця з МВФ
# 31 - штрафи АМКУ?
# 11 - курс валют
# 8 - критика премій і зарплат членів НКРЕКП
# 9 - Ціни на продукти, зокрема Великодній кошик

economics_topic_dict = {
    3: 'Курс валют',
    39: 'Курс валют',
    14: 'Ціни та тарифи на газ',
    13: 'Карантинні виплати ФОПам',
    35: 'Пенсії та соціальні виплати',
#     24: 'Зелена енергетика',
    21: 'Співпраця України з МВФ',
#     8: 'Критика НКРЕКП, зокрема високих зарплат та премій',
    4: 'Ціни на продукти, зокрема на Великодній кошик',
#     24: 'Ціни на пальне в Україні',
#     4: 'Авторинок в Україні та світі',
#     34: 'Екомодернізація української промисловості',
#     1: 'Ціни на нафту'
#     8: 'бюджет'
#     16: 'банки',
#     14: 'підтримка під час карантину'
#     11: 'вагони/уз/інфрастуктура'
}


part['topic'] = part.topic_40_stem.apply(lambda x: economics_topic_dict.get(x, 'економіка_решта'))

#### Світ

In [93]:
world_topic_dict = {
#         0 : 'киргизстан',
#         17 : 'білорусь',
#         10 : 'білорусь',
#         14 : 'отруєння навального'
#     14: 'теракти у франції',
#     1: 'карантин у світі',
#     13: 'карантин у світі'
#     8: 'інциденти у світі',
#     1: 'інциденти у світі',
#     5: 'нобелівська премія'
    3: 'політика сша'
}

part['topic'] = part.topic_20_lemm.apply(lambda x: world_topic_dict.get(x, 'світ_решта'))

#### Міжнародна політика

In [289]:
# 30 - Дипломатичний конфлікт між Чехією та Росією
# 36 - Заяви України та Росії щодо можливої зустрічі Зеленського і Путіна
# 32 - Стягнення Росією військ до кордону і реакція міжнародної спільноти
# 34 - висилка дипломатів, українських
# 42 - Події в Білорусі: заяви Лукашенка, Тихановської та реакція інших країн
# 13 - Визнання Байденом геноциду вірмен і відповідь Туреччини

# 27 - обговорення вступу україни в нато

international_politics_topic_dict = {
    12: 'Стягнення Росією військ до кордону і реакція міжнародної спільноти',
    22: 'Стягнення Росією військ до кордону і реакція міжнародної спільноти',
#     32: 'Стягнення Росією військ до кордону і реакція міжнародної спільноти',
#     30: 'Дипломатичний конфлікт між Чехією та Росією',
#     34: 'Взаємна висилка дипломатів Україною та Росією',
    0: 'Висилка російських дипломатів з різних країн та відповідь Росії',
#     36: 'Заяви України та Росії щодо можливої зустрічі Зеленського і Путіна',
#     18: 'Будівництво Північного потоку-2 і реакція різних країн',
#     2: 'Події в Білорусі: заяви Лукашенка, Тихановської та реакція інших країн',
#     13: 'Визнання Байденом геноциду вірмен і відповідь Туреччини',
#     35: "Стан Олексія Навального та реакція міжнародної спільноти",
#     13: 'Збиття Іраном літака МАУ: нові заяви та звинувачення',
#     3: 'Виведення з Афганістану військ США та інших країн',
#     29: 'Співпраця України з НАТО, заяви щодо потенційного членства в альянсі',
#     14: 'Викрадення екссудді Чауса',
}



part['topic'] = part.topic_30_stem.apply(lambda x: international_politics_topic_dict.get(x, 'міжн_пол_решта'))

#### Коронавірус

In [850]:
covid_topic_dict = {
#     50: 'Початок вакцинації вакцинами від Pfizer і SinoVac в Україні',
#     14: 'Початок вакцинації вакцинами від Pfizer і SinoVac в Україні'
    0: "Розробка української вакцини проти Covid-19",
    1: "Розробка української вакцини проти Covid-19",
    4: "Розробка української вакцини проти Covid-19"
#     20: 'Нові штами коронавірусу, зокрема «британський» і «південноафриканський»',
#     9: 'Спалах Covid-19 в Індії',
#     21: 'Смерті й побічні реакції після вакцинації',
#     9: "Зв'язок між вакцинами (зокрема AstraZeneca та Johnson&Johnson) і тробмозом",
#     21: 'Ефективність вакцин',
#     26: "Паспорти вакцинації та COVID-сертифікати"
#     35: "Зв'язок між вакцинами (зокрема AstraZeneca та Johnson&Johnson) і тробмозом",
#     35: 'Карантинні рекомендації до святкування Великодня',
#     12: "Sputnik-V"
#     24: "Закриття Covid-відділеннь в Ізраїлі"
#     18: 'Національний план вакцинації',
#     39: 'Темпи вакцинації в Україні',
}


part['topic'] = part.topic_40_stem.apply(lambda x: covid_topic_dict.get(x, 'ковід_решта'))

#### Інциденти

In [140]:
#     4 - аварія поїзда УЗ
#     3 - справа стерненка, акція під ОПУ
#     17 - порушення карантинних обмежень
#     27 - контрабанда
#     37 - справа стерненка

# 15 - звільнення антоненка
# 31 - справа стерненка і Справи учасників акції на підтримку Стерненка під ОПУ

incidents_topic_dict={
    1: 'Справи учасників акції на підтримку Стерненка під ОПУ',
#     4: 'Аварія поїзда "Київ – Запоріжжя"',
#     29: 'Порушення карантинних обмежень',
    0: 'Справа Стерненка',
#     39: 'Справа "ПВК Семенченка"',
#     5: 'Стрілянина в поїзді "Костянтинівка-Київ"',
#     2: 'Справа Юрія Зонтова – брата Павла Вовка',
#     11: 'Викрадення судді Чауса',
#     38: 'Затримання в окупованому Криму та справи кримських політв\'язнів',
#     12: 'Антитерористичні навчання СБУ, нацгвардії, нацполіції'
}


part['topic'] = part.topic_50_stem.apply(lambda x: incidents_topic_dict.get(x, 'інциденти_решта'))

In [461]:
karantyn_topic_dict = {
    12: 'заклади освіти',
    10: 'заклади освіти',
#     14: 'карантинні зони/нові обмеження',
#     11: 'карантинні зони/нові обмеження',
    5: 'карантинні зони/нові обмеження',
    13: 'карантинні зони/нові обмеження',
    9: 'карантинні зони/нові обмеження',
    6: 'дотримання обмежень в регіонах',
    1: 'дотримання обмежень в регіонах'
#     13: 'карантин у світі'
#     2: 'транспортне сполучення в карантин'
}

part['topic'] = part.topic_15_stem.apply(lambda x: karantyn_topic_dict.get(x, 'карантин_решта'))

### Save to df and file for analisys

In [144]:
part['topic'].value_counts()

Стягнення Росією військ до кордону і загострення на Донбасі             660
Співпраця України з НАТО, заяви щодо потенційного членства в альянсі    366
Name: topic, dtype: int64

In [3]:
# part.loc[(part.topic_40_stem==4)&(part.all_text.str.contains('лукашенк', flags=re.I)), 'topic_40_stem']=50
# part[(part.topic_40_stem==1)&((part.all_text.str.contains('накопичен|стягнен|відведе|ескалац|скупч')))].all_text.iloc[64]


In [24]:
model_data = {
    'topic_term_dists': topic_term_dists, 
    'doc_topic_dists': doc_topic_dists,
    'doc_lengths': doc_lengths,
    'vocab': vocab,
    'term_frequency': term_frequency
}

vis_data = pyLDAvis.prepare(**model_data)
pyLDAvis.display(vis_data)